Modules nécessaires : 

In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler

from HiggsML.ingestion import Ingestion
from HiggsML.datasets import download_dataset
from sample_code_submission.neural_network import NeuralNetwork
from sys import path
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
from itertools import product
from numpy.random import RandomState
import warnings
import os
import sys

warnings.filterwarnings("ignore")

Charger les données : 

In [2]:
data = download_dataset(
    "blackSwan_data"
)  # change to "blackSwan_data" for the actual data

# load train set
data.load_train_set()
data_set = data.get_train_set()

2025-06-04 09:48:08,887 - HiggsML.datasets     - INFO     - Handling as dataset name: blackSwan_data
2025-06-04 09:48:08,887 - HiggsML.datasets     - INFO     - Current working directory: c:\Users\marwa\Desktop\Neural Network\Higgs_collaboration_A
2025-06-04 09:48:08,899 - HiggsML.datasets     - INFO     - Total rows: 2000000
2025-06-04 09:48:08,904 - HiggsML.datasets     - INFO     - Test size: 600000
2025-06-04 09:48:08,987 - HiggsML.datasets     - INFO     - Selected train size: 1400000
2025-06-04 09:48:10,077 - HiggsML.datasets     - INFO     - Data loaded successfully


In [3]:
target = data_set["labels"]
weights = data_set["weights"]
detailed_label = data_set["detailed_labels"]
keys = np.unique(detailed_label)

Direction

In [4]:
root_dir = os.getcwd()
print("Root directory is", root_dir)
submission_dir = os.path.join(root_dir, "sample_code_submission")

# The directory where results and other outputs from the participant's code will be written
output_dir = os.path.join(root_dir, "sample_result_submission")

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

Root directory is c:\Users\marwa\Desktop\Neural Network\Higgs_collaboration_A


Import Submission Model

In [5]:
path.append(submission_dir)
from model import Model

Testing Settings

In [6]:
TEST_SETTINGS = {
    "systematics": {  # Systematics to use
        "tes": False,  # tau energy scale
        "jes": False,  # jet energy scale
        "soft_met": False,  # soft term in MET
        "ttbar_scale": False,  # W boson scale factor
        "diboson_scale": False,  # Diboson scale factor
        "bkg_scale": False,  # Background scale factor
    },
    "num_pseudo_experiments": 25,  # Number of pseudo-experiments to run per set
    "num_of_sets": 25,  # Number of sets of pseudo-experiments to run
}

RANDOM_SEED = 42


In [7]:
test_settings = TEST_SETTINGS.copy()

random_state = np.random.RandomState(RANDOM_SEED)
test_settings["ground_truth_mus"] = (
    random_state.uniform(0.1, 3, test_settings["num_of_sets"])
).tolist()

random_settings_file = os.path.join(output_dir, "test_settings.json")
with open(random_settings_file, "w") as f:
    json.dump(test_settings, f)

Boucle sur "Ingestion" en faisant varier epochs

Fonction bouclée: 

In [11]:
L_epochs = np.linspace(2, 3, 2)
sigmax = 5000
epochs_max = 1

Significance

In [ ]:

def optimization(train_data): 
    L_epochs = np.linspace(2, 3, 2)
    sigmax = 5000
    epochs_max = 1
    for k in range(len(L_epochs)):
        path.append(submission_dir)
        ingestion = Ingestion(data)
        # initialize submission
        ingestion.init_submission(Model, "NN")
        ingestion.model.model.epochs = L_epochs[k]
        print(ingestion.model.model.epochs)
        ingestion.fit_submission()
        sig1 = ingestion.model.model.significance_2(test_labels=ingestion.model.training_set["labels"],test_weights=ingestion.model.training_set["weights"])
        #visualisation des données :
        if sig1>sigmax: 
            sigmax = sig1
            epochs_max = L_epochs[k]
    return sigmax, epochs_max



Lancement de la boucle

In [19]:
optimization(data_set)

2025-06-04 09:52:58,301 - HiggsML.ingestion    - INFO     - Initializing Submmited Model
2025-06-04 09:52:58,304 - HiggsML.datasets     - INFO     - Selected train size: 5000
2025-06-04 09:52:58,675 - HiggsML.datasets     - INFO     - Data loaded successfully
2025-06-04 09:52:58,709 - HiggsML.datasets     - INFO     - Selected train size: 5000


Training Data:  (5000, 28)
Training Labels:  (5000,)
Training Weights:  (5000,)
sum_signal_weights:  613.8731328026427
sum_bkg_weights:  105105.12686719734


2025-06-04 09:52:59,057 - HiggsML.datasets     - INFO     - Data loaded successfully
2025-06-04 09:52:59,077 - HiggsML.datasets     - INFO     - Selected train size: 5000



Valid Data:  (5000, 28)
Valid Labels:  (5000,)
Valid Weights:  (5000,)
sum_signal_weights:  701.3039205106508
sum_bkg_weights:  105017.69607948934


2025-06-04 09:52:59,408 - HiggsML.datasets     - INFO     - Data loaded successfully



Holdout Data:  (5000, 28)
Holdout Labels:  (5000,)
Holdout Weights:  (5000,)
sum_signal_weights:  688.3363944674295
sum_bkg_weights:  105030.6636055326
 
 
Training Data:  (5000, 28)
DEBUG: model_type = 'NN'
 Model is NN


AttributeError: 'Ingestion' object has no attribute 'epochs'